# Segment the time series into 1 minute sequences for each user

In [1]:
from Funcs.Utility import *
import numpy as np
import pandas as pd
from typing import Dict, Callable, Union, Tuple, List, Optional, Iterable
from datetime import timedelta as td
from scipy import stats
import ray
import warnings
import time

In [2]:
def _safe_na_check(_v):
    _is_nan_inf = False
    
    try:
        _is_nan_inf = np.isnan(_v) or np.isinf(_v)
    except:
        _is_nan_inf = False
    
    return _is_nan_inf or _v is None

In [3]:
import os
import cloudpickle

DATA = load(os.path.join(PATH_INTERMEDIATE, 'proc.pkl'))
LABELS_PROC = pd.read_csv(os.path.join(PATH_INTERMEDIATE, 'proc', 'LABELS_PROC.csv'), index_col=['pcode','timestamp'],parse_dates=True)

In [11]:
# import pandas as pd
# import ray
# import logging
# logging.basicConfig(level=logging.DEBUG)

# RESAMPLE_S = {
#     'ACC_AXX': 0.25,
#     'ACC_AXY': 0.25,
#     'ACC_AXZ': 0.25,
#     'ACC_MAG': 0.25,
#     'EDA': 0.5,
# }

# @ray.remote
# def segment_data(pcode, sensor_data, label_data):
#     user_labels = label_data.loc[pcode]
#     if isinstance(user_labels, pd.Series):
#         user_labels = user_labels.to_frame().T

#     labeled_sequences = []
#     unlabeled_sequences = []

#     for sensor_type, data in sensor_data.items():
#         resampled_data = data.resample('T').asfreq()
#         if isinstance(resampled_data, pd.Series):
#             resampled_data = resampled_data.to_frame()

#         for time, row in resampled_data.iterrows():
#             sequences = {sensor_type: data.loc[time:time + pd.Timedelta(minutes=1)] for sensor_type, data in sensor_data.items()}

#             # Filter to future times and find the nearest time index
#             future_labels = user_labels[user_labels.index > time]
#             if future_labels.empty:
#                 unlabeled_sequences.append(sequences)
#                 continue
#             else:
#                 time_differences = (future_labels.index - time).total_seconds()
#                 abs_time_differences = abs(pd.Series(time_differences, index=future_labels.index))
#                 nearest_future_time = abs_time_differences.idxmin()
#                 label_row = future_labels.loc[nearest_future_time]
            
#             if pd.isna(label_row['duration']):
#                 overlapping_labels = user_labels[(user_labels.index >= time) & 
#                                     (user_labels.index - pd.Timedelta(minutes= 1) < time)]
#             else:
#                 overlapping_labels = user_labels[(user_labels.index >= time) & 
#                                                 (user_labels.index - pd.Timedelta(minutes= label_row['duration']) < time)]
#             if not overlapping_labels.empty:
#                 label = overlapping_labels.iloc[-1]['stress_fixed']
#                 labeled_sequences.append((sequences, label))
#             else:
#                 unlabeled_sequences.append(sequences)
#     print(f"Finished resampling data for {pcode}...")
#     return labeled_sequences, unlabeled_sequences

# with on_ray(num_cpus=12):
    
#     segmented_data = []
#     for pcode in LABELS_PROC.index.get_level_values('pcode').unique():
#         print(f"Resampling data for {pcode}...")
#         resampled_sensor_data = {}
#         for sensor_type, data in DATA.items():
#             if pcode in data.index.get_level_values('pcode'):
#                 resample_interval = RESAMPLE_S.get(sensor_type, 1)
#                 user_data = data.loc[pcode]
#                 resampled_sensor_data[sensor_type] = user_data.resample(f'{resample_interval}S').interpolate(method='linear').dropna()

#         segmented_data.append(segment_data.remote(pcode, resampled_sensor_data, LABELS_PROC))

#     results = ray.get(segmented_data)
#     labeled_sequences = [(sequences, label) for labeled_sequences, _ in results for sequences, label in labeled_sequences]
#     unlabeled_sequences = [sequences for _, unlabeled_sequences in results for sequences in unlabeled_sequences]

#     labeled_df = pd.concat([pd.concat(list(sequences.values()), axis=1) for sequences, _ in labeled_sequences], ignore_index=True)
#     unlabeled_df = pd.concat([pd.concat(list(sequences.values()), axis=1) for sequences in unlabeled_sequences], ignore_index=True)

DEBUG:filelock:Attempting to acquire lock 140358849091904 on /tmp/ray/session_2023-12-05_16-36-28_198389_180383/ports_by_node.json.lock
DEBUG:filelock:Lock 140358849091904 acquired on /tmp/ray/session_2023-12-05_16-36-28_198389_180383/ports_by_node.json.lock
DEBUG:filelock:Attempting to release lock 140358849091904 on /tmp/ray/session_2023-12-05_16-36-28_198389_180383/ports_by_node.json.lock
DEBUG:filelock:Lock 140358849091904 released on /tmp/ray/session_2023-12-05_16-36-28_198389_180383/ports_by_node.json.lock
DEBUG:filelock:Attempting to acquire lock 140374630609488 on /tmp/ray/session_2023-12-05_16-36-28_198389_180383/ports_by_node.json.lock
DEBUG:filelock:Lock 140374630609488 acquired on /tmp/ray/session_2023-12-05_16-36-28_198389_180383/ports_by_node.json.lock
DEBUG:filelock:Attempting to release lock 140374630609488 on /tmp/ray/session_2023-12-05_16-36-28_198389_180383/ports_by_node.json.lock
DEBUG:filelock:Lock 140374630609488 released on /tmp/ray/session_2023-12-05_16-36-28_19

Resampling data for P01...
Resampling data for P02...
Resampling data for P03...
Resampling data for P04...
Resampling data for P05...
Resampling data for P06...
Resampling data for P07...
Resampling data for P08...
Resampling data for P09...
Resampling data for P10...
(segment_data pid=187840) Finished resampling data for P01...
Resampling data for P11...
Resampling data for P12...
Resampling data for P13...
(segment_data pid=187841) Finished resampling data for P02...
Resampling data for P14...
Resampling data for P15...
(segment_data pid=187843) Finished resampling data for P04...
(segment_data pid=187842) Finished resampling data for P03... [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
Resampling data for P16...
(segment_data pid=187845) Finished resampling data for P07...
Resampling data for P18...


2023-12-05 16:40:13,088	WARNING worker.py:2006 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: 47f241abe42ff0287d577da2285a4af8053190f801000000 Worker ID: b166a79893eae26f82684dcb477c038bf3868c8165c2c80dce98aee6 Node ID: 94e0007f7148eeb737b711a6c5f7e33c80337d82ddd6b977127848d7 Worker IP address: 143.248.57.67 Worker port: 42059 Worker PID: 187844 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.


(segment_data pid=187836) Finished resampling data for P06...
Resampling data for P19...


2023-12-05 16:40:27,865	WARNING worker.py:2006 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: fb9e97d30b01d87cbb02feafbbb7c3b9547cf42d01000000 Worker ID: 86c58c9190c529f2d5a0190f29a0eb1a9fb960f4d9b24baa77b6e770 Node ID: 94e0007f7148eeb737b711a6c5f7e33c80337d82ddd6b977127848d7 Worker IP address: 143.248.57.67 Worker port: 35477 Worker PID: 187841 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.
(raylet) [2023-12-05 16:40:30,551 E 187726 187726] (raylet) node_manager.cc:3084: 39 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons 

Resampling data for P20...


2023-12-05 16:40:43,680	WARNING worker.py:2006 -- A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: 6f73b2e181cb6a3be6bf8bb8eefdac3c60efb7f001000000 Worker ID: 4597ae6b7341c825cd23c3a414a0b0a8aff3f019124aebda96198118 Node ID: 94e0007f7148eeb737b711a6c5f7e33c80337d82ddd6b977127848d7 Worker IP address: 143.248.57.67 Worker port: 37595 Worker PID: 187843 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker unexpectedly exits with a connection error code 2. End of file. There are some potential root causes. (1) The process is killed by SIGKILL by OOM killer due to high memory usage. (2) ray stop --force is called. (3) The worker is crashed unexpectedly due to SIGSEGV or other unexpected errors.


KeyboardInterrupt: 

In [14]:
import pandas as pd
import ray
import logging

logging.basicConfig(level=logging.DEBUG)

RESAMPLE_S = {
    'ACC_AXX': 0.25,
    'ACC_AXY': 0.25,
    'ACC_AXZ': 0.25,
    'ACC_MAG': 0.25,
    'EDA': 0.5,
}

@ray.remote
def segment_sensor_data(pcode, sensor_type, sensor_data, label_data):
    user_labels = label_data.loc[pcode]
    if isinstance(user_labels, pd.Series):
        user_labels = user_labels.to_frame().T

    labeled_sequences = []
    unlabeled_sequences = []

    resampled_data = sensor_data.resample('T').asfreq()
    if isinstance(resampled_data, pd.Series):
        resampled_data = resampled_data.to_frame()

    for time, row in resampled_data.iterrows():
        sequence = {
        sensor_type: sensor_data.loc[time:time + pd.Timedelta(minutes=1)],
        'pcode': pcode  # Add the pcode to the sequence
        }

        future_labels = user_labels[user_labels.index > time]
        if not future_labels.empty:
            time_differences = (future_labels.index - time).total_seconds()
            abs_time_differences = abs(pd.Series(time_differences, index=future_labels.index))
            nearest_future_time = abs_time_differences.idxmin()
            label_row = future_labels.loc[nearest_future_time]

            if pd.isna(label_row['duration']):
                overlapping_labels = user_labels[(user_labels.index >= time) & 
                                    (user_labels.index - pd.Timedelta(minutes=1) < time)]
            else:
                overlapping_labels = user_labels[(user_labels.index >= time) & 
                                                (user_labels.index - pd.Timedelta(minutes=label_row['duration']) < time)]

            if not overlapping_labels.empty:
                label = overlapping_labels.iloc[-1]['stress_fixed']
                sequence['label'] = label  # Add the label to the sequence
                labeled_sequences.append(sequence)
            else:
                sequence['label'] = None
                unlabeled_sequences.append(sequence)
        else:
            unlabeled_sequences.append(sequence)

    return labeled_sequences, unlabeled_sequences


with on_ray():

    segmented_data = []
    for pcode in LABELS_PROC.index.get_level_values('pcode').unique():
        for sensor_type, data in DATA.items():
            if pcode in data.index.get_level_values('pcode'):
                print(f"Resampling {sensor_type} data for {pcode}...")
                resample_interval = RESAMPLE_S.get(sensor_type, 1)
                user_data = data.loc[pcode]
                resampled_sensor_data = user_data.resample(f'{resample_interval}S').interpolate(method='linear').dropna()

                segmented_data.append(segment_sensor_data.remote(pcode, sensor_type, resampled_sensor_data, LABELS_PROC))

    results = ray.get(segmented_data)

    # Aggregate labeled and unlabeled sequences
    all_labeled_sequences = [item for sublist in results for item in sublist[0]]
    all_unlabeled_sequences = [item for sublist in results for item in sublist[1]]

    # Concatenate sequences into dataframes
    labeled_df = pd.concat([pd.concat(sequence.values(), axis=1) for sequence in all_labeled_sequences], ignore_index=True)
    unlabeled_df = pd.concat([pd.concat(sequence.values(), axis=1) for sequence in all_unlabeled_sequences], ignore_index=True)

DEBUG:filelock:Attempting to acquire lock 140362910611824 on /tmp/ray/session_2023-12-05_16-47-47_502859_180383/ports_by_node.json.lock
DEBUG:filelock:Lock 140362910611824 acquired on /tmp/ray/session_2023-12-05_16-47-47_502859_180383/ports_by_node.json.lock
DEBUG:filelock:Attempting to release lock 140362910611824 on /tmp/ray/session_2023-12-05_16-47-47_502859_180383/ports_by_node.json.lock
DEBUG:filelock:Lock 140362910611824 released on /tmp/ray/session_2023-12-05_16-47-47_502859_180383/ports_by_node.json.lock
DEBUG:filelock:Attempting to acquire lock 140362910611824 on /tmp/ray/session_2023-12-05_16-47-47_502859_180383/ports_by_node.json.lock
DEBUG:filelock:Lock 140362910611824 acquired on /tmp/ray/session_2023-12-05_16-47-47_502859_180383/ports_by_node.json.lock
DEBUG:filelock:Attempting to release lock 140362910611824 on /tmp/ray/session_2023-12-05_16-47-47_502859_180383/ports_by_node.json.lock
DEBUG:filelock:Lock 140362910611824 released on /tmp/ray/session_2023-12-05_16-47-47_50

2023-12-05 16:47:50,196	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Resampling ACC_AXX data for P01...
Resampling ACC_AXY data for P01...
Resampling ACC_AXZ data for P01...
Resampling ACC_MAG data for P01...
Resampling AML data for P01...
Resampling CAL data for P01...
Resampling DST_DST data for P01...
Resampling DST_PAC data for P01...
Resampling DST_SPD data for P01...
Resampling EDA data for P01...
Resampling STP data for P01...
Resampling ACC_AXX data for P02...
Resampling ACC_AXY data for P02...
Resampling ACC_AXZ data for P02...
Resampling ACC_MAG data for P02...
Resampling AML data for P02...
Resampling CAL data for P02...
Resampling DST_DST data for P02...
Resampling DST_PAC data for P02...
Resampling DST_SPD data for P02...
Resampling EDA data for P02...
Resampling STP data for P02...
Resampling ACC_AXX data for P03...
Resampling ACC_AXY data for P03...
Resampling ACC_AXZ data for P03...
Resampling ACC_MAG data for P03...
Resampling AML data for P03...
Resampling CAL data for P03...
Resampling DST_DST data for P03...
Resampling DST_PAC data f

(raylet) Spilled 12261 MiB, 735 objects, write throughput 16973 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


Resampling DST_PAC data for P25...
Resampling DST_SPD data for P25...
Resampling EDA data for P25...
Resampling STP data for P25...


...

Resampling ACC_AXY data for P26...
Resampling ACC_AXZ data for P26...
Resampling ACC_MAG data for P26...
Resampling AML data for P26...
Resampling CAL data for P26...
Resampling DST_DST data for P26...
Resampling DST_PAC data for P26...
Resampling DST_SPD data for P26...
Resampling EDA data for P26...
Resampling STP data for P26...
Resampling ACC_AXX data for P28...
Resampling ACC_AXY data for P28...
Resampling ACC_AXZ data for P28...
Resampling ACC_MAG data for P28...
Resampling AML data for P28...
Resampling CAL data for P28...
Resampling DST_DST data for P28...
Resampling DST_PAC data for P28...
Resampling DST_SPD data for P28...
Resampling EDA data for P28...
Resampling STP data for P28...
Resampling ACC_AXX data for P29...
Resampling ACC_AXY data for P29...
Resampling ACC_AXZ data for P29...
Resampling ACC_MAG data for P29...
Resampling AML data for P29...
Resampling CAL data for P29...
Resampling DST_DST data for P29...
Resampling DST_PAC data for P29...
Resampling DST_SPD data f

(raylet) Spilled 16517 MiB, 999 objects, write throughput 4652 MiB/s.


Resampling STP data for P34...
Resampling ACC_AXX data for P35...
Resampling ACC_AXY data for P35...
Resampling ACC_AXZ data for P35...
Resampling ACC_MAG data for P35...
Resampling AML data for P35...
Resampling CAL data for P35...
Resampling DST_DST data for P35...
Resampling DST_PAC data for P35...
Resampling DST_SPD data for P35...
Resampling EDA data for P35...
Resampling STP data for P35...
Resampling ACC_AXX data for P36...
Resampling ACC_AXY data for P36...
Resampling ACC_AXZ data for P36...
Resampling ACC_MAG data for P36...
Resampling AML data for P36...
Resampling CAL data for P36...
Resampling DST_DST data for P36...
Resampling DST_PAC data for P36...
Resampling DST_SPD data for P36...
Resampling EDA data for P36...
Resampling STP data for P36...
Resampling ACC_AXX data for P37...
Resampling ACC_AXY data for P37...
Resampling ACC_AXZ data for P37...
Resampling ACC_MAG data for P37...
Resampling AML data for P37...
Resampling CAL data for P37...
Resampling DST_DST data for P

(raylet) Spilled 33079 MiB, 2046 objects, write throughput 2396 MiB/s.


Resampling ACC_MAG data for P68...
Resampling AML data for P68...
Resampling CAL data for P68...
Resampling DST_DST data for P68...
Resampling DST_PAC data for P68...
Resampling DST_SPD data for P68...
Resampling EDA data for P68...
Resampling STP data for P68...
Resampling ACC_AXX data for P69...
Resampling ACC_AXY data for P69...
Resampling ACC_AXZ data for P69...
Resampling ACC_MAG data for P69...
Resampling AML data for P69...
Resampling CAL data for P69...
Resampling DST_DST data for P69...
Resampling DST_PAC data for P69...
Resampling DST_SPD data for P69...
Resampling EDA data for P69...
Resampling STP data for P69...
Resampling ACC_AXX data for P70...
Resampling ACC_AXY data for P70...
Resampling ACC_AXZ data for P70...
Resampling ACC_MAG data for P70...
Resampling AML data for P70...
Resampling CAL data for P70...
Resampling DST_DST data for P70...
Resampling DST_PAC data for P70...
Resampling DST_SPD data for P70...
Resampling EDA data for P70...
Resampling STP data for P70..

In [ ]:
labeled_df.to_csv(os.path.join(PATH_INTERMEDIATE, 'proc', 'labeled_sequences.csv'), index=False)
unlabeled_df.to_csv(os.path.join(PATH_INTERMEDIATE, 'proc', 'unlabeled_sequences.csv'), index=False)